In [5]:
import ppsim as pp
from dataclasses import dataclass
import dataclasses
import numpy as np
from matplotlib import pyplot as plt
import pickle
%matplotlib widget
import ipywidgets as widgets

# DSD oscillator 

Below is an implementation of a rock-paper-scissors (RPS) oscillator using DNA strand displacement. The "formal" CRN for the RPS oscillator is 

- A+B &rarr; 2B
- B+C &rarr; 2C
- C+A &rarr; 2A

Each reaction is implemented by 8 lower-level reactions describing DNA interactions. For details, see Fig. 1 in http://dx.doi.org/10.1126/science.aal2052 (bioRxiv verion: https://www.biorxiv.org/content/10.1101/138420v2).

In [9]:
from ppsim import species

# Fig. 1 in https://www.biorxiv.org/content/10.1101/138420v2.full.pdf
# A+B --> 2B
# B+C --> 2C
# C+A --> 2A

# signal species (represent formal species in formal CRN above)
# index indicates whether it was the first or second product of a previous reaction
b1, b2, c1, c2, a1, a2 = species('b1  b2  c1  c2  a1  a2')

signal_species = [b1, b2, c1, c2, a1, a2]

# fuel species react step
react_a_b_b1, back_a_b = species('react_a_b_b1  back_a_b')
react_b_c_c1, back_b_c = species('react_b_c_c1  back_b_c')
react_c_a_a1, back_c_a = species('react_c_a_a1  back_c_a')

react_species = [react_a_b_b1, react_b_c_c1, react_c_a_a1]
back_species = [back_a_b, back_b_c, back_c_a]

# fuel species produce step
produce_b_b1_b2, helper_b_b2 = species('produce_b_b1_b2  helper_b_b2')
produce_c_c1_c2, helper_c_c2 = species('produce_c_c1_c2  helper_c_c2')
produce_a_a1_a2, helper_a_a2 = species('produce_a_a1_a2  helper_a_a2')

produce_species = [produce_b_b1_b2, produce_c_c1_c2, produce_a_a1_a2]
helper_species = [helper_b_b2, helper_c_c2, helper_a_a2]
fuel_species = react_species + produce_species

# intermediate species
flux_b_b1, flux_c_c1, flux_a_a1 = species('flux_b_b1  flux_c_c1  flux_a_a1')
reactint_a1_b_b1, reactint_b1_c_c1, reactint_c1_a_a1 = species('reactint_a1_b_b1  reactint_b1_c_c1  reactint_c1_a_a1') 
reactint_a2_b_b1, reactint_b2_c_c1, reactint_c2_a_a1 = species('reactint_a2_b_b1  reactint_b2_c_c1  reactint_c2_a_a1') 
productint_b_b1_b2, productint_c_c1_c2, productint_a_a1_a2 = species('productint_b_b1_b2  productint_c_c1_c2  productint_a_a1_a2')

flux_species = [flux_b_b1, flux_c_c1, flux_a_a1]
reactint_species = [reactint_a1_b_b1, reactint_b1_c_c1, reactint_c1_a_a1,
                    reactint_a2_b_b1, reactint_b2_c_c1, reactint_c2_a_a1]
produceint_species = [productint_b_b1_b2, productint_c_c1_c2, productint_a_a1_a2]

# waste species react step
waste_a1_b1, waste_a1_b2, waste_a2_b1, waste_a2_b2 = species('waste_a1_b1  waste_a1_b2  waste_a2_b1  waste_a2_b2')
waste_b1_c1, waste_b1_c2, waste_b2_c1, waste_b2_c2 = species('waste_b1_c1  waste_b1_c2  waste_b2_c1  waste_b2_c2')
waste_c1_a1, waste_c1_a2, waste_c2_a1, waste_c2_a2 = species('waste_c1_a1  waste_c1_a2  waste_c2_a1  waste_c2_a2')

# waste species produce step
waste_b_b1_b2, waste_c_c1_c2, waste_a_a1_a2 = species('waste_b_b1_b2  waste_c_c1_c2  waste_a_a1_a2')

waste_species = [waste_a1_b1, waste_a1_b2, waste_a2_b1, waste_a2_b2,
                 waste_b1_c1, waste_b1_c2, waste_b2_c1, waste_b2_c2,
                 waste_c1_a1, waste_c1_a2, waste_c2_a1, waste_c2_a2,
                 waste_b_b1_b2, waste_c_c1_c2, waste_a_a1_a2]

# DSD reactions implementing formal CRN
# A+B --> 2B
ab_react_rxns = [
    a1 + react_a_b_b1 | back_a_b + reactint_a1_b_b1,
    a2 + react_a_b_b1 | back_a_b + reactint_a2_b_b1,
    reactint_a1_b_b1 + b1 >> waste_a1_b1 + flux_b_b1, # typo in Fig. 1; these rxns irreversible
    reactint_a1_b_b1 + b2 >> waste_a1_b2 + flux_b_b1, #
    reactint_a2_b_b1 + b1 >> waste_a2_b1 + flux_b_b1, #
    reactint_a2_b_b1 + b2 >> waste_a2_b2 + flux_b_b1, #
]
ab_produce_rxns = [
    flux_b_b1 + produce_b_b1_b2 | b1 + productint_b_b1_b2,
    helper_b_b2 + productint_b_b1_b2 >> waste_b_b1_b2 + b2,
]
ab_rxns = ab_react_rxns + ab_produce_rxns

# B+C --> 2C
bc_react_rxns = [
    b1 + react_b_c_c1 | back_b_c + reactint_b1_c_c1,
    b2 + react_b_c_c1 | back_b_c + reactint_b2_c_c1,
    reactint_b1_c_c1 + c1 >> waste_b1_c1 + flux_c_c1,
    reactint_b1_c_c1 + c2 >> waste_b1_c2 + flux_c_c1,
    reactint_b2_c_c1 + c1 >> waste_b2_c1 + flux_c_c1,
    reactint_b2_c_c1 + c2 >> waste_b2_c2 + flux_c_c1,
]
bc_produce_rxns = [
    flux_c_c1 + produce_c_c1_c2 | c1 + productint_c_c1_c2,
    helper_c_c2 + productint_c_c1_c2 >> waste_c_c1_c2 + c2,
]
bc_rxns = bc_react_rxns + bc_produce_rxns

# C+A --> 2A
ca_react_rxns = [
    c1 + react_c_a_a1 | back_c_a + reactint_c1_a_a1,
    c2 + react_c_a_a1 | back_c_a + reactint_c2_a_a1,
    reactint_c1_a_a1 + a1 >> waste_c1_a1 + flux_a_a1,
    reactint_c1_a_a1 + a2 >> waste_c1_a2 + flux_a_a1,
    reactint_c2_a_a1 + a1 >> waste_c2_a1 + flux_a_a1,
    reactint_c2_a_a1 + a2 >> waste_c2_a2 + flux_a_a1,
]
ca_produce_rxns = [
    flux_a_a1 + produce_a_a1_a2 | a1 + productint_a_a1_a2,
    helper_a_a2 + productint_a_a1_a2 >> waste_a_a1_a2 + a2,
]
ca_rxns = ca_react_rxns + ca_produce_rxns

all_rps_dsd_rxns = ab_rxns + bc_rxns + ca_rxns

all_species = signal_species + \
              react_species + \
              back_species + \
              produce_species + \
              helper_species + \
              flux_species + \
              reactint_species + \
              produceint_species + \
              waste_species

def aux(state):
    if state in react_species:
        return 'react'
    if state in produce_species:
        return 'produce'
    if state in waste_species:
        return 'waste'
    if state in helper_species:
        return 'helper'
    
def abc(state):
    if state in signal_species:
        return state.name[0]

In [10]:
from ppsim import Simulation, RateConstantUnits, concentration_to_count

uL = 10 ** -6  # 1 uL (microliter)
nL = 10 ** -9
nM = 10 ** -9  # 1 nM (nanomolar)

k = 1e6  # forward rate constant in mass-action units
r = 1e6  # reverse rate constant in mass-action units
for rxn in all_rps_dsd_rxns:
    rxn.k(k, units=RateConstantUnits.mass_action)
    if rxn.reversible:
        rxn.r(r, units=RateConstantUnits.mass_action)

vol = 10 * nL

# scale time to make simulations take less time
time_scaling = 1
vol /= time_scaling

react_conc = 100 * nM
back_conc = 100 * nM
helper_conc = 75 * nM
produce_conc = 100 * nM
a1_conc = 11 * nM
b1_conc = 10 * nM
c1_conc = 3 * nM

# this factor scales all concentrations
conc_factor = 1

react_count = concentration_to_count(react_conc * conc_factor, vol)
back_count = concentration_to_count(back_conc * conc_factor , vol)
helper_count = concentration_to_count(helper_conc* conc_factor, vol)
produce_count = concentration_to_count(produce_conc* conc_factor, vol)
a1_count = concentration_to_count(a1_conc* conc_factor, vol)
b1_count = concentration_to_count(b1_conc* conc_factor, vol)
c1_count = concentration_to_count(c1_conc* conc_factor, vol)

init_config_react = {specie: react_count for specie in react_species}
init_config_back = {specie: back_count for specie in back_species}
init_config_helper = {specie: helper_count for specie in helper_species}
init_config_produce = {specie: produce_count for specie in produce_species}

init_config = {a1: a1_count, b1: b1_count, c1: c1_count}
init_config.update(init_config_react)
init_config.update(init_config_back)
init_config.update(init_config_helper)
init_config.update(init_config_produce)

sim = Simulation(init_config=init_config, rule=all_rps_dsd_rxns, volume=vol, time_units='seconds')

In [ ]:
%time sim.run()
hp = pp.HistoryPlotter(abc, update_time=1)
hp2 = pp.HistoryPlotter(aux, update_time=1)
sim.add_snapshot(hp)
sim.add_snapshot(hp2)
pickle.dump(sim, open( "crn_data/dsd_experiment.p", "wb" ) )

In [2]:
sim = pickle.load( open( "crn_data/dsd_experiment.p", "rb" ))

ModuleNotFoundError: No module named 'ppsim.ppsim'

In [4]:
pp.write_stochkit_file('dsd_example.xml',all_rps_dsd_rxns, init_config, vol)

In [64]:
import random
def sample_one(config):
    x = random.randrange(0, sum(config))
    i = 0
    while x > 0:
        x -= config[i]
        i += 1
    return i-1

def sample_state(sim):
    return sim.state_list[sample_one(sim.config_array)]

def sample_null_probability(sim):
    num_trials = 100000
    num_nulls = 0
    for _ in range(num_trials):
        a, b = sample_state(sim), sample_state(sim)
        if sim.rule(a,b) is None or set(sim.rule(a,b)) == set((a, b)):
            num_nulls += 1
    return num_nulls / num_trials

In [65]:
sample_null_probability(sim)

0.99645

In [22]:
sim.history

,a1,a2,b1,b2,back_a_b,back_b_c,back_c_a,c1,c2,flux_a_a1,...,waste_b1_c1,waste_b1_c2,waste_b2_c1,waste_b2_c2,waste_b_b1_b2,waste_c1_a1,waste_c1_a2,waste_c2_a1,waste_c2_a2,waste_c_c1_c2
time (continuous),,,,,,,,,,,,,,,,,,,,,
0,3011,0,6022,0,60221,60221,60221,3011,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2728,0,5461,0,60504,60780,60497,2733,0,0,...,2,0,0,0,0,0,0,0,0,0
2,2522,0,5038,0,60707,61201,60729,2499,0,3,...,5,0,0,0,0,3,0,0,0,0
3,2316,0,4677,0,60909,61555,60900,2326,1,7,...,8,0,0,0,0,7,0,0,0,1
4,2154,0,4415,0,61070,61812,61083,2134,1,8,...,17,0,0,0,0,10,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4339,1804,1715,717,689,82090,84984,84270,1487,1503,122,...,7900,6081,5289,4935,20206,6417,5459,5669,5276,24075
4340,1796,1728,717,690,82094,84988,84287,1469,1511,125,...,7900,6081,5289,4936,20209,6418,5461,5671,5277,24081
4341,1746,1695,724,689,82186,84985,84282,1485,1504,122,...,7901,6081,5289,4936,20210,6419,5462,5672,5277,24084
